In [14]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import urlretrieve
import re
import os

In [2]:
content_url = "https://freemidi.org/"
html_doc = urlopen(content_url+"genre").read()

In [3]:
sp = BeautifulSoup(html_doc)
sp

/opt/conda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


<!DOCTYPE html>
<html><!-- InstanceBegin template="/Templates/newborn.dwt.php" codeOutsideHTMLIsLocked="false" -->
<head>
<meta charset="utf-8"/>
<!-- InstanceBeginEditable name="doctitle" -->
<title> Free Midi</title>
<!-- InstanceEndEditable -->
<link href="/images/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/images/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/images/favicon/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/images/favicon/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/images/favicon/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/images/favicon/apple-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/images/favicon/apple-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<link href="/images/favicon/apple-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="/images/favi

# ジャンルのパスを取得

In [4]:
genres = sp.find_all("div", {"class":"genre-big-ones"})
genres_href = [[tag["href"] for tag in genre.find_all("a")] for genre in genres]
genres_href

[['genre-rock', 'genre-pop', 'genre-hip-hop-rap', 'genre-rnb-soul'],
 ['genre-classical', 'genre-country', 'genre-jazz', 'genre-blues'],
 ['genre-dance-eletric', 'genre-folk', 'genre-punk', 'genre-newage'],
 ['genre-reggae-ska', 'genre-metal', 'genre-disco', 'genre-bluegrass']]

# ジャンルのパスから各ジャンルへのスクレイピンング

genere_artist format is as following

```
{genre_href: {artist:{song_name: href}}}
```

In [7]:
genre_artist = {}
selected_genre = ["genre-hip-hop-rap", "genre-disco"]
for hrefs in genres_href:
    for href in hrefs:
        if not href in selected_genre:
            continue
        print(href)
        content = {}
        url = content_url + href
        html_doc = BeautifulSoup(urlopen(url).read())
        artist_hrefs = html_doc.find_all("div", {"class": "genre-link-text"})
        for artist_href in artist_hrefs:
            artist_href_ = artist_href.a.get("href")
            a_html_doc = BeautifulSoup(urlopen(content_url+artist_href_).read())
            content[artist_href.a.string] = {re.sub("\r\n\s{2,}", "", artist_href.a.string):a.get("href") for a in a_html_doc.find_all("a", {"itemprop":"url"})[1:]}
        genre_artist[href] = content

genre-hip-hop-rap


/opt/conda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


genre-disco


# 各アーティストの楽曲のダウンロードメウ

In [23]:
#print(genre_artist)
for key, item in genre_artist.items():
    if not os.path.exists(key):
        os.mkdir(key)
        
    for artist_n, song_dict in item.items():
        if not os.path.exists(key+"/"+artist_n):
            os.mkdir(key+"/"+artist_n)
            
        for song_n, song_href in song_dict.items():
            try:
                href = BeautifulSoup(urlopen(content_url+song_href).read()).find("a", {"id":"downloadmidi"}).get("href")
                urlretrieve(content_url+href, key+"/"+artist_n+"/"+song_n+".mid")
            except:
                continue

/opt/conda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
